In [16]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv("data/API_PA.NUS.PRVT.PP_DS2_en_csv_v2_1869577.csv", skiprows=3)

In [18]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', 'Unnamed: 65'],
      dtype='object')

In [19]:
df = df[["Country Name", "Country Code", "2019"]]
df = df.rename(columns={"Country Name": "Country"})
df.head()

,Country,Country Code,2019
0,Aruba,ABW,1.533552
1,Afghanistan,AFG,17.134464
2,Angola,AGO,NaN
3,Albania,ALB,50.993347
4,Andorra,AND,NaN


In [20]:
df = df.sort_values(by="Country").reset_index(drop=True)
df = df.dropna()

In [21]:
curr = pd.read_csv("countries.csv")

In [22]:
curr.columns

Index(['name', 'name_fr', 'ISO3166-1-Alpha-2', 'ISO3166-1-Alpha-3',
       'ISO3166-1-numeric', 'ITU', 'MARC', 'WMO', 'DS', 'Dial', 'FIFA', 'FIPS',
       'GAUL', 'IOC', 'currency_alphabetic_code', 'currency_country_name',
       'currency_minor_unit', 'currency_name', 'currency_numeric_code',
       'is_independent'],
      dtype='object')

In [23]:
curr = curr[["name", "ITU", "currency_alphabetic_code", "currency_name"]]
curr = curr.rename(columns={"name":"Country", "currency_alphabetic_code":"Currency", "currency_name": "Currency Name"})

In [24]:
s1 = pd.merge(df, curr, on=["Country"])

In [25]:
final = pd.merge(df, s1, on=["Country"])

In [26]:
final.columns = final.columns.str.replace(r'_x$', '')
final.columns = final.columns.str.replace(r'_y$', '')
final = final.loc[:, ~final.columns.duplicated()]

In [27]:
final.head()

,Country,Country Code,2019,ITU,Currency,Currency Name
0,Afghanistan,AFG,17.134464,AFG,AFN,Afghani
1,Albania,ALB,50.993347,ALB,ALL,Lek
2,Algeria,DZA,38.944652,ALG,DZD,Algerian Dinar
3,Armenia,ARM,166.832568,ARM,AMD,Armenian Dram
4,Aruba,ABW,1.533552,ABW,AWG,Aruban Florin


In [28]:
final = final.drop(columns=["ITU"])
final = final[["Country", "Currency", "Currency Name", "2019"]]
final.head()

,Country,Currency,Currency Name,2019
0,Afghanistan,AFN,Afghani,17.134464
1,Albania,ALL,Lek,50.993347
2,Algeria,DZD,Algerian Dinar,38.944652
3,Armenia,AMD,Armenian Dram,166.832568
4,Aruba,AWG,Aruban Florin,1.533552


In [29]:
final.shape

(130, 4)

In [30]:
def convert_by_ppp(source_country, target_country, salary):
    return salary / list(final[final["Country"]==source_country]["2019"])[0] * list(final[final["Country"]==target_country]["2019"])[0]

In [31]:
val = convert_by_ppp("United States", "United Kingdom", 150000)
round(val, 2)

116329.65

In [32]:
final.to_csv("data.csv", index=False)